# Traveling Salesman Problem

## Objective and Prerequisites

In this example, you’ll learn how to tackle one of the most famous combinatorial optimization problems in existence: the Traveling Salesman Problem (TSP). The goal of the TSP – to find the shortest possible route that visits each city once and returns to the original city – is simple, but solving the problem is a complex and challenging endeavor. We’ll show you how to do it!

This modeling example is at the advanced level, where we assume that you know Python and the Gurobi Python API and that you have advanced knowledge of building mathematical optimization models. Typically, the objective function and/or constraints of these examples are complex or require advanced features of the Gurobi Python API.

**Download the Repository** <br />
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip).

## Motivation

The Traveling Salesman Problem (TSP) is one of the most famous combinatorial optimization problems. This problem is very easy to explain, but very complicated to solve – even for instances with a small number of cities. More detailed information on the TSP can be found in the book The Traveling Salesman Problem: A Computational Study [1], or at the TSP home page [2]. If you are interested in the history and mathematical background of the TSP, we recommend that you watch the video by William Cook [3].

The origin of the traveling salesman problem is not very clear; it is mentioned in an 1832 manual for traveling salesman, which included example tours of 45 German cities but was not formulated as a mathematical problem. However, in the 1800s, mathematicians William Rowan Hamilton and Thomas Kirkman devised mathematical formulations of the problem.

It seems that the general form of the Traveling Salesman Problem was first studied by Karl Menger in Vienna and Harvard in the 1930s.

The problem became more and more popular in the 1950s and 1960s. In particular, George Dantzig, D. Ray Fulkerson, and Selmer M. Johnson at the RAND Corporation solved the 48-state problem by formulating it as a linear programming problem. The methods they described in their paper on this topic set the foundation for future work in combinatorial optimization, especially highlighting the importance of cutting planes.

In the early 1970s, the concept of P vs. NP problems created excitement in the theoretical computer science community. In 1972, Richard Karp demonstrated that the Hamiltonian cycle problem was NP-complete, implying that the traveling salesman problem was NP-hard.

Increasingly sophisticated codes led to rapid increases in the sizes of the traveling salesman problems solved. Dantzig, Fulkerson, and Johnson had solved a 48-city instance of the problem in 1954. Martin Grötschel more than doubled this 23 years later, solving a 120-city instance in 1977. Harlan Crowder and Manfred W. Padberg again more than doubled this in just 3 years, with a 318-city solution.

In 1987, rapid improvements were made, culminating in a 2,392-city solution by Padberg and Giovanni Rinaldi. In the following two decades, great strides were made with David L. Applegate, Robert E. Bixby, Vasek Chvátal, & William J. Cook solving a 3,308-city instance in 1992, a 7,397-city instance in 1994, a 24,978-city instance in 2004, and an 85,900-city instance in 2006 – which is the largest 2-D Euclidean TSP instance ever solved. William Cook et. al. wrote a program called Concorde TSP Solver for solving the TSP [4]. Concorde is a computer code for the symmetric TSP and some related network optimization problems. The code is written in the ANSI C programming language and it has been used to obtain the optimal solutions to the full set of 110 TSPLIB instances, the largest instance is a 109,399 node 3-D “star” instance.

The continued interest in the TSP can be explained by its success as a general engine of discovery and a steady stream of new applications. Some of the general applications of the TSP are as follows:
* Scheduling and routing problems.
* Genome sequencing.
* Drilling problems.
* Aiming telescopes and x-rays.
* Data clustering.
* Machine scheduling.

We use this classic combinatorial optimization problem to demonstrate how Gurobi can be used to easily and effectively solve small-sized problem instances of the TSP. However, in order to be able to solve larger instances, one needs more sophisticated techniques – such as those implemented in the Concord TSP Solver.

## Problem Description
The TSP can be defined as follows: for a given list of cities and the distances between each pair of them, we want to find the shortest possible route that goes to each city once and returns to the origin city.

There is a class of Traveling Salesman Problems that assumes that the distance of going from city $i$ to city $j$  is the same as going form city $j$ to city $i$, this type of Traveling Salesman Problem  is also known as the symmetric Traveling Salesman Problem. In this example, we use Euclidean distances, but the TSP model formulation is valid independent of the way in which the individual distances are determined.


## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a MIP formulation of the TSP that identifies the shortest route that goes to all the cities once and returns to the origin city.

## TSP Model Formulation

### Sets and Indices
$i, j \in Capitals $: indices and set of US capital cities.

$\text{Pairings}= \{(i,j) \in Capitals \times Capitals \}$: Set of allowed pairings

$S \subset Capitals$: A subset of the set of US capital cities.

$G = (Capitals, Pairings)$: A graph where the set $Capitals$ defines the set of nodes and the set $Pairings$ defines the set of edges.

### Parameters

$d_{i, j} \in \mathbb{R}^+$: Distance from capital city $i$ to capital city $j$, for all $(i, j) \in Pairings$.

Notice that the distance from capital city $i$ to capital city $j$ is the same as the distance from capital city $j$ to capital city $i$, i.e. $d_{i, j} = d_{j, i}$. For this reason, this TSP is also called the symmetric Traveling Salesman Problem.

### Decision Variables
$x_{i, j} \in \{0, 1\}$: This variable is equal to 1, if we decide to connect city $i$ with city $j$. Otherwise, the decision variable is equal to zero.

### Objective Function
- **Shortest Route**. Minimize the total distance of a route. A route is a sequence of capital cities where the salesperson visits each city only once and returns to the starting capital city.

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in \text{Pairings}}d_{i,j} \cdot x_{i,j}
\tag{0}
\end{equation}

### Constraints
- **Symmetry Constraints**. For each edge $(i,j)$, ensure that the city capitals $i$ and $j$ are connected, if the former is visited immediately before or after visiting the latter.

\begin{equation}
x_{i, j} = x_{j, i} \quad \forall (i, j) \in Pairings
\tag{1}
\end{equation}

- **Entering and leaving a capital city**. For each capital city $i$, ensure that this city is connected to two other cities.

\begin{equation}
\sum_{(i,j) \in \text{Pairings}}x_{i,j} = 2 \quad \forall  i \in Capitals
\tag{2}
\end{equation}

- **Subtour elimination**. These constraints ensure that for any subset of cities $S$ of the set of $Capitals$, there is no cycle. That is, there is no route that visits all the cities in the subset and returns to the origin city.

\begin{equation}
\sum_{(i \neq j) \in S}x_{i,j} \leq |S|-1 \quad \forall  S \subset  Capitals
\tag{3}
\end{equation}

- **Remark**. In general, if the number of cities of the TSP is $n$, then the possible number of routes is n\!.
Since there are an exponential number of constraints ($2^{n} - 2$) to eliminate cycles, we use lazy constraints to dynamically eliminate those cycles.

## Python Implementation

Consider a salesperson that needs to visit customers at each state capital of the continental US. The salesperson wants to identify the shortest route that goes to all the state capitals.

This modeling example requires the following libraries that are not part of the standard Python distribution:
* **folium**: to create maps.
* **gurobipy**: provides Gurobi algorithms to solve MIP models.


### Reading Input Data
The capital names and coordinates are read from a json file.

In [1]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 65.0 MB/s eta 0:00:00


In [12]:
import json

# Read capital names and coordinates from json file
try:
  capitals_json = json.load(open('capitals.json'))
# when running locally the following lines can be omitted
except:
  import urllib.request
  url = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/traveling_salesman/capitals.json'
  data = urllib.request.urlopen(url).read()
  capitals_json = json.loads(data)

capitals = []
coordinates = {}
for state in capitals_json:
    if state not in ['AK', 'HI']:
      capital = capitals_json[state]['capital']
      capitals.append(capital)
      coordinates[capital] = (float(capitals_json[state]['lat']), float(capitals_json[state]['long']))

### Data computation
The following function calculates the distance for each pair of state capitals. Since we are solving the _symmetric_ traveling salesman problem, we use _combinations_ of cities.

In [13]:
import math
from itertools import combinations

# Compute pairwise distance matrix

def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}

### Model Code
We now write the model for the TSP, by defining decision variables, constraints, and objective function. Because this is the _symmetric_ traveling salesman problem, we can make it more efficient by setting the _object_ x[j,i] to x[i,j], instead of a constraint.

In [14]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7 & Gurobi 9.0.0

m = gp.Model()

# Variables: is city 'i' adjacent to city 'j' on the tour?
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# Symmetric direction: use dict.update to alias variable with new key
vars.update({(j,i):vars[i,j] for i,j in vars.keys()})

# Constraints: two edges incident to each city
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

Restricted license - for non-production use only - expires 2026-11-23


### Callback Definition
Subtour constraints prevent multiple loops in a TSP tour. Because there are an exponential number of these constraints, we don't want to add them all to the model. Instead, we use a callback function to find violated subtour constraints and add them to the model as lazy constraints.

In [23]:
# Callback - use lazy constraints to eliminate sub-tours

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    n = len(capitals)
    visited = [False] * n
    cities_to_index = {city: i for i, city in enumerate(capitals)}
    index_to_cities = {i: city for i, city in enumerate(capitals)}
    adj = [[] for _ in range(n)]

    for i, j in edges:
        u = cities_to_index[i]
        v = cities_to_index[j]
        adj[u].append(v)
        adj[v].append(u)

    shortest_cycle = []

    def dfs(u, current_cycle):
        visited[u] = True
        current_cycle.append(index_to_cities[u])

        for v in adj[u]:
            if not visited[v]:
                dfs(v, current_cycle)
        return current_cycle

    for i in range(n):
        if not visited[i]:
            cycle = dfs(i, [])
            if not shortest_cycle or len(cycle) < len(shortest_cycle):
                shortest_cycle = cycle
    return shortest_cycle

## Solve the model

In [16]:
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
LazyConstraints  1

Optimize a model with 48 rows, 1128 columns and 2256 nonzeros
Model fingerprint: 0x63641a38
Variable types: 0 continuous, 1128 integer (1128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 48 rows, 1128 columns, 2256 nonzeros
Variable types: 0 continuous, 1128 integer (1128 binary)

Root relaxation: objective 1.611858e+02, 72 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     

## Analysis

We retrieve the optimal solution of the TSP and verify that the optimal route (or tour) goes to all the cities and returns to the origin city.

In [20]:
# Retrieve solution

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capitals)

print("Optimal tour:", tour)

Optimal tour: ['Montgomery', 'Tallahassee', 'Atlanta', 'Columbia', 'Raleigh', 'Richmond', 'Annapolis', 'Dover', 'Trenton', 'Hartford', 'Providence', 'Boston', 'Augusta', 'Concord', 'Montpelier', 'Albany', 'Harrisburg', 'Charleston', 'Columbus', 'Lansing', 'Madison', 'Saint Paul', 'Des Moines', 'Topeka', 'Lincoln', 'Pierre', 'Bismarck', 'Cheyenne', 'Denver', 'Salt Lake City', 'Helana', 'Boise', 'Olympia', 'Salem', 'Sacramento', 'Carson City', 'Phoenix', 'Santa Fe', 'Oklahoma City', 'Austin', 'Baton Rouge', 'Jackson', 'Little Rock', 'Jefferson City', 'Springfield', 'Indianapolis', 'Frankfort', 'Nashville']


In [21]:
# Get the objective value (total distance)
total_distance = m.objVal

print(f"Total distance of the optimal tour: {total_distance}")

Total distance of the optimal tour: 175.4490370197638


The optimal route is displayed in the following map.

In [25]:
# Map the solution

import folium

map = folium.Map(location=[40,-95], zoom_start = 4)

points = []
for city in tour:
  points.append(coordinates[city])
points.append(points[0])

print("Geographical coordinates for each city in the tour:", points)

folium.PolyLine(points).add_to(map)

map

Geographical coordinates for each city in the tour: [(32.361538, -86.279118), (30.4518, -84.27277), (33.76, -84.39), (34.0, -81.035), (35.771, -78.638), (37.54, -77.46), (38.972945, -76.501157), (39.161921, -75.526755), (40.221741, -74.756138), (41.767, -72.677), (41.82355, -71.422132), (42.2352, -71.0275), (44.323535, -69.765261), (43.220093, -71.549127), (44.26639, -72.57194), (42.659829, -73.781339), (40.269789, -76.875613), (38.349497, -81.633294), (39.962245, -83.000647), (42.7335, -84.5467), (43.074722, -89.384444), (44.95, -93.094), (41.590939, -93.620866), (39.04, -95.69), (40.809868, -96.675345), (44.367966, -100.336378), (46.813343, -100.779004), (41.145548, -104.802042), (39.7391667, -104.984167), (40.7547, -111.892622), (46.595805, -112.027031), (43.613739, -116.237651), (47.042418, -122.893077), (44.931109, -123.029159), (38.555605, -121.468926), (39.160949, -119.753877), (33.448457, -112.073844), (35.667231, -105.964575), (35.482309, -97.534994), (30.266667, -97.75), (30.

In [9]:
m.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment


## Conclusions

The Traveling Salesman Problem (TSP) is the most popular combinatorial optimization problem. This problem is very easy to explain, although it is very complicated to solve. The largest TSP problem solved has 85,900 cities. The TSP is a source of discovery for new approaches to solve complex combinatorial optimization problems and has led to many applications.

In this modeling example, we have shown how to formulate the symmetric Traveling Salesman Problem as a MIP problem. We also showed how to dynamically eliminate subtours by using lazy constraints.

## References

[1] D. L. Applegate, R. E. Bixby, V. Chvatal and W. J. Cook , The Traveling Salesman Problem: A Computational Study, Princeton University Press, Princeton, 2006.

[2] http://www.math.uwaterloo.ca/tsp/index.html

[3] https://www.youtube.com/watch?v=q8nQTNvCrjE&t=35s

[4] http://www.math.uwaterloo.ca/tsp/concorde.html

Copyright © 2020 Gurobi Optimization, LLC

# Task
Generate a markdown file summarizing the Traveling Salesperson Problem (TSP) example, including the problem description, model formulation, Python implementation details, subtour elimination explanation, results, and conclusions.

## Summarize problem description

### Subtask:
Extract and summarize the problem description from the notebook.


**Reasoning**:
Extract and summarize the problem description from the notebook based on the instructions.



In [26]:
problem_description_cell = '''
## Problem Description
The TSP can be defined as follows: for a given list of cities and the distances between each pair of them, we want to find the shortest possible route that goes to each city once and returns to the origin city.

There is a class of Traveling Salesman Problems that assumes that the distance of going from city $i$ to city $j$  is the same as going form city $j$ to city $i$, this type of Traveling Salesman Problem  is also known as the symmetric Traveling Salesman Problem. In this example, we use Euclidean distances, but the TSP model formulation is valid independent of the way in which the individual distances are determined.
'''

# Summarize the problem description
summary = "The Traveling Salesman Problem (TSP) aims to find the shortest possible route that visits each city in a given list exactly once and returns to the starting city. This example focuses on the symmetric TSP, where the distance between two cities is the same in both directions, and uses Euclidean distances."

print(summary)

The Traveling Salesman Problem (TSP) aims to find the shortest possible route that visits each city in a given list exactly once and returns to the starting city. This example focuses on the symmetric TSP, where the distance between two cities is the same in both directions, and uses Euclidean distances.


## Summarize model formulation

### Subtask:
Extract and summarize the mathematical model formulation, including sets, parameters, variables, objective function, and constraints.


**Reasoning**:
Extract and summarize the mathematical model formulation from the identified markdown cell.



In [27]:
model_formulation_cell = """
## TSP Model Formulation

### Sets and Indices
$i, j \in Capitals $: indices and set of US capital cities.

$\text{Pairings}= \{(i,j) \in Capitals \times Capitals \}$: Set of allowed pairings

$S \subset Capitals$: A subset of the set of US capital cities.

$G = (Capitals, Pairings)$: A graph where the set $Capitals$ defines the set of nodes and the set $Pairings$ defines the set of edges.

### Parameters

$d_{i, j} \in \mathbb{R}^+$: Distance from capital city $i$ to capital city $j$, for all $(i, j) \in Pairings$.

Notice that the distance from capital city $i$ to capital city $j$ is the same as the distance from capital city $j$ to capital city $i$, i.e. $d_{i, j} = d_{j, i}$. For this reason, this TSP is also called the symmetric Traveling Salesman Problem.

### Decision Variables
$x_{i, j} \in \{0, 1\}$: This variable is equal to 1, if we decide to connect city $i$ with city $j$. Otherwise, the decision variable is equal to zero.

### Objective Function
- **Shortest Route**. Minimize the total distance of a route. A route is a sequence of capital cities where the salesperson visits each city only once and returns to the starting capital city.

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in \text{Pairings}}d_{i,j} \cdot x_{i,j}
\tag{0}
\end{equation}

### Constraints
- **Symmetry Constraints**. For each edge $(i,j)$, ensure that the city capitals $i$ and $j$ are connected, if the former is visited immediately before or after visiting the latter.

\begin{equation}
x_{i, j} = x_{j, i} \quad \forall (i, j) \in Pairings
\tag{1}
\end{equation}

- **Entering and leaving a capital city**. For each capital city $i$, ensure that this city is connected to two other cities.

\begin{equation}
\sum_{(i,j) \in \text{Pairings}}x_{i,j} = 2 \quad \forall  i \in Capitals
\tag{2}
\end{equation}

- **Subtour elimination**. These constraints ensure that for any subset of cities $S$ of the set of $Capitals$, there is no cycle. That is, there is no route that visits all the cities in the subset and returns to the origin city.

\begin{equation}
\sum_{(i \neq j) \in S}x_{i,j} \leq |S|-1 \quad \forall  S \subset  Capitals
\tag{3}
\end{equation}

- **Remark**. In general, if the number of cities of the TSP is $n$, then the possible number of routes is n\!.
Since there are an exponential number of constraints ($2^{n} - 2$) to eliminate cycles, we use lazy constraints to dynamically eliminate those cycles.
"""

# Summarize the model formulation
summary = """
**Mathematical Model Formulation:**

**Sets and Indices:**
- `Capitals`: Set of US capital cities, indexed by `i` and `j`.
- `Pairings`: Set of allowed pairs of cities `(i, j)`.
- `S`: A subset of `Capitals`.
- `G`: A graph representing cities as nodes and pairings as edges.

**Parameters:**
- `d_i,j`: Distance between city `i` and city `j`. Distances are symmetric (`d_i,j = d_j,i`).

**Decision Variables:**
- `x_i,j`: Binary variable, 1 if city `i` is connected to city `j` in the tour, 0 otherwise.

**Objective Function:**
- Minimize the total distance of the tour: `Min Z = Sum(d_i,j * x_i,j) for all (i,j) in Pairings`.

**Constraints:**
- **Symmetry:** `x_i,j = x_j,i` for all `(i,j)` in Pairings (enforced implicitly in the Python code by aliasing variables).
- **Degree Constraint:** Each city must be connected to exactly two other cities: `Sum(x_i,j for all j connected to i) = 2` for all `i` in Capitals.
- **Subtour Elimination:** For any subset of cities `S`, the number of connections within the subset must be less than or equal to `|S|-1`. This prevents cycles that do not include all cities and is handled dynamically using lazy constraints.
"""

print(summary)


**Mathematical Model Formulation:**

**Sets and Indices:**
- `Capitals`: Set of US capital cities, indexed by `i` and `j`.
- `Pairings`: Set of allowed pairs of cities `(i, j)`.
- `S`: A subset of `Capitals`.
- `G`: A graph representing cities as nodes and pairings as edges.

**Parameters:**
- `d_i,j`: Distance between city `i` and city `j`. Distances are symmetric (`d_i,j = d_j,i`).

**Decision Variables:**
- `x_i,j`: Binary variable, 1 if city `i` is connected to city `j` in the tour, 0 otherwise.

**Objective Function:**
- Minimize the total distance of the tour: `Min Z = Sum(d_i,j * x_i,j) for all (i,j) in Pairings`.

**Constraints:**
- **Symmetry:** `x_i,j = x_j,i` for all `(i,j)` in Pairings (enforced implicitly in the Python code by aliasing variables).
- **Degree Constraint:** Each city must be connected to exactly two other cities: `Sum(x_i,j for all j connected to i) = 2` for all `i` in Capitals.
- **Subtour Elimination:** For any subset of cities `S`, the number of connecti

<>:5: SyntaxWarning: invalid escape sequence '\i'
<>:5: SyntaxWarning: invalid escape sequence '\i'
/tmp/ipython-input-1164089624.py:5: SyntaxWarning: invalid escape sequence '\i'
  $i, j \in Capitals $: indices and set of US capital cities.


## Describe python implementation

### Subtask:
Briefly explain the key steps in the Python implementation, including data loading, distance calculation, and model creation.


**Reasoning**:
Explain the Python implementation steps including data loading, distance calculation, and model creation as described in the subtask.



In [28]:
# Explanation of Python Implementation

implementation_explanation = """
## Python Implementation Details

The Python implementation of the TSP model involves several key steps:

1.  **Reading Input Data:**
    -   The state capital names and their geographical coordinates (latitude and longitude) are loaded from a JSON file (`capitals.json`).
    -   The code iterates through the loaded JSON data.
    -   It filters out Alaska ('AK') and Hawaii ('HI') as this example focuses on the continental US capitals.
    -   The capital names are stored in a list called `capitals`.
    -   A dictionary called `coordinates` is created to store each capital's name as the key and a tuple of its latitude and longitude as the value.

2.  **Data Computation (Distance Calculation):**
    -   A `distance` function is defined to calculate the Euclidean distance between two cities given their coordinates.
    -   The `itertools.combinations` function is used to generate all unique pairs of cities from the `capitals` list. Since this is a symmetric TSP, the distance from city A to city B is the same as from city B to city A, so we only need to calculate each pair's distance once.
    -   A dictionary called `dist` is created to store the calculated distances. The keys are tuples representing the city pairs `(city1, city2)`, and the values are the computed distances.

3.  **Model Creation:**
    -   A Gurobi model is initialized using `gp.Model()`.
    -   Binary decision variables (`vars`) are added to the model using `m.addVars()`. These variables correspond to the city pairings (`dist.keys()`) and represent whether an edge exists between the cities in the optimal tour (`vtype=GRB.BINARY`). The objective function coefficients for these variables are set to the calculated distances (`obj=dist`).
    -   To handle the symmetry constraint implicitly, the `vars` dictionary is updated so that accessing `vars[j, i]` returns the same variable as `vars[i, j]` for each pair `(i, j)`.
    -   Degree constraints (`cons`) are added using `m.addConstrs()`. For each capital city `c`, this constraint ensures that the sum of the `vars` connected to `c` (using `vars.sum(c, '*')`) is exactly 2, meaning each city must have exactly two edges connected to it in the tour.
"""

print(implementation_explanation)


## Python Implementation Details

The Python implementation of the TSP model involves several key steps:

1.  **Reading Input Data:**
    -   The state capital names and their geographical coordinates (latitude and longitude) are loaded from a JSON file (`capitals.json`).
    -   The code iterates through the loaded JSON data.
    -   It filters out Alaska ('AK') and Hawaii ('HI') as this example focuses on the continental US capitals.
    -   The capital names are stored in a list called `capitals`.
    -   A dictionary called `coordinates` is created to store each capital's name as the key and a tuple of its latitude and longitude as the value.

2.  **Data Computation (Distance Calculation):**
    -   A `distance` function is defined to calculate the Euclidean distance between two cities given their coordinates.
    -   The `itertools.combinations` function is used to generate all unique pairs of cities from the `capitals` list. Since this is a symmetric TSP, the distance from city 

## Explain subtour elimination

### Subtask:
Summarize the concept of subtour elimination and the use of lazy constraints and the callback function.


**Reasoning**:
Summarize the concept of subtour elimination, lazy constraints, and the callback function as requested.



In [29]:
# Explanation of Subtour Elimination and Lazy Constraints

subtour_explanation = """
## Subtour Elimination and Lazy Constraints

**1. What are Subtours and why eliminate them?**

In the context of the Traveling Salesperson Problem (TSP), a "subtour" is a cycle within the selected edges that does not include all the cities. A valid TSP "tour" must visit *every* city exactly once and return to the starting city, forming a single, large cycle. If the solution contains one or more smaller cycles (subtours) that don't encompass all cities, it's not a complete tour and therefore an invalid solution to the TSP. Subtour elimination is necessary to ensure that the optimization model finds a single, connected tour that includes all cities.

**2. Subtour Elimination Constraints:**

Mathematically, subtour elimination constraints ensure that for any subset of cities $S$ (where $S$ is not empty and not the set of all cities), the number of selected edges connecting cities *within* that subset $S$ is at most $|S|-1$. This prevents the formation of cycles that are smaller than the complete tour. The constraint formulation is:
$$ \sum_{(i, j) \in S, i \neq j} x_{i, j} \leq |S| - 1 \quad \forall S \subset Capitals, 2 \leq |S| < |Capitals| $$

**3. Why not add all subtour constraints upfront?**

The number of possible subsets of cities grows exponentially with the number of cities ($2^n - 2$ for $n$ cities, excluding the empty set and the full set). For even moderately sized problems, adding all these constraints to the model upfront would result in an extremely large model that is computationally intractable to build and solve.

**4. Lazy Constraints in Gurobi:**

Gurobi's lazy constraints provide an efficient way to handle constraints that are only likely to be violated in the optimal or near-optimal solutions. Instead of adding all potential constraints at the beginning, lazy constraints are added *dynamically* during the optimization process. When the solver finds an integer feasible solution, it calls a user-defined callback function. This function checks if the solution violates any lazy constraints (in this case, subtour elimination constraints). If a violation is found (i.e., a subtour exists), the corresponding constraint is added to the model, and the solver continues. This process repeats until a solution is found that is feasible for all (explicit and lazy) constraints.

**5. Role of the Callback Function (`subtourelim`):**

The `subtourelim` function in the provided code serves as the callback. When Gurobi finds a potential integer solution (`where == GRB.Callback.MIPSOL`), the callback is triggered. Inside the callback:
- It retrieves the values of the decision variables (`model._vars`) in the current solution.
- It identifies the edges that are "selected" in this solution (where the variable value is greater than 0.5).
- It then calls the `subtour` helper function to find the shortest cycle within these selected edges.
- If the length of the found cycle (`tour`) is less than the total number of cities (`len(capitals)`), it means a subtour has been detected.
- For each pair of cities `(i, j)` within the detected subtour, a lazy subtour elimination constraint is added to the model using `model.cbLazy()`. This constraint forces the number of selected edges within this specific subtour to be at most `len(tour) - 1`, thereby eliminating this subtour in future solutions.

**6. The `subtour` Helper Function:**

The `subtour(edges)` function takes a list of selected edges as input and identifies the shortest cycle (subtour) within those edges. It does this by:
- Representing the selected edges as an adjacency list for a graph.
- Using a Depth First Search (DFS) algorithm to traverse the graph starting from an unvisited city.
- The DFS explores connected components. If a connected component is found that does not include all cities, it represents a subtour.
- The function returns the cities forming the shortest detected cycle. This cycle is then used by the `subtourelim` callback to formulate and add the necessary lazy constraint.
"""

print(subtour_explanation)


## Subtour Elimination and Lazy Constraints

**1. What are Subtours and why eliminate them?**

In the context of the Traveling Salesperson Problem (TSP), a "subtour" is a cycle within the selected edges that does not include all the cities. A valid TSP "tour" must visit *every* city exactly once and return to the starting city, forming a single, large cycle. If the solution contains one or more smaller cycles (subtours) that don't encompass all cities, it's not a complete tour and therefore an invalid solution to the TSP. Subtour elimination is necessary to ensure that the optimization model finds a single, connected tour that includes all cities.

**2. Subtour Elimination Constraints:**

Mathematically, subtour elimination constraints ensure that for any subset of cities $S$ (where $S$ is not empty and not the set of all cities), the number of selected edges connecting cities *within* that subset $S$ is at most $|S|-1$. This prevents the formation of cycles that are smaller than the 

<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1429121.py:13: SyntaxWarning: invalid escape sequence '\s'
  $$ \sum_{(i, j) \in S, i \neq j} x_{i, j} \leq |S| - 1 \quad \forall S \subset Capitals, 2 \leq |S| < |Capitals| $$


## Present results

### Subtask:
Include the optimal tour found and the total distance. Mention the visualization of the tour on the map.


**Reasoning**:
Format the optimal tour and total distance and mention the map visualization.



In [36]:
# Format the results

results_summary = f"""
## Results

The Gurobi optimizer found the optimal tour for the Traveling Salesperson Problem covering the continental US capitals.

**Optimal Tour:**
{tour}

**Total Distance of the Optimal Tour:**
{total_distance:.4f}

The optimal tour is also visualized on an interactive map.
"""

print(results_summary)


## Results

The Gurobi optimizer found the optimal tour for the Traveling Salesperson Problem covering the continental US capitals.

**Optimal Tour:**
['Montgomery', 'Tallahassee', 'Atlanta', 'Columbia', 'Raleigh', 'Richmond', 'Annapolis', 'Dover', 'Trenton', 'Hartford', 'Providence', 'Boston', 'Augusta', 'Concord', 'Montpelier', 'Albany', 'Harrisburg', 'Charleston', 'Columbus', 'Lansing', 'Madison', 'Saint Paul', 'Des Moines', 'Topeka', 'Lincoln', 'Pierre', 'Bismarck', 'Cheyenne', 'Denver', 'Salt Lake City', 'Helana', 'Boise', 'Olympia', 'Salem', 'Sacramento', 'Carson City', 'Phoenix', 'Santa Fe', 'Oklahoma City', 'Austin', 'Baton Rouge', 'Jackson', 'Little Rock', 'Jefferson City', 'Springfield', 'Indianapolis', 'Frankfort', 'Nashville']

**Total Distance of the Optimal Tour:**
175.4490

The optimal tour is also visualized on an interactive map.



**Reasoning**:
The previous command failed due to a `SyntaxError`. Regenerate the code block for formatting the results, ensuring the f-string is correctly terminated.



In [37]:
# Format the results

results_summary = f"""
## Results

The Gurobi optimizer found the optimal tour for the Traveling Salesperson Problem covering the continental US capitals.

**Optimal Tour:**
{tour}

**Total Distance of the Optimal Tour:**
{total_distance:.4f}

The optimal tour is also visualized on an interactive map.
"""

print(results_summary)


## Results

The Gurobi optimizer found the optimal tour for the Traveling Salesperson Problem covering the continental US capitals.

**Optimal Tour:**
['Montgomery', 'Tallahassee', 'Atlanta', 'Columbia', 'Raleigh', 'Richmond', 'Annapolis', 'Dover', 'Trenton', 'Hartford', 'Providence', 'Boston', 'Augusta', 'Concord', 'Montpelier', 'Albany', 'Harrisburg', 'Charleston', 'Columbus', 'Lansing', 'Madison', 'Saint Paul', 'Des Moines', 'Topeka', 'Lincoln', 'Pierre', 'Bismarck', 'Cheyenne', 'Denver', 'Salt Lake City', 'Helana', 'Boise', 'Olympia', 'Salem', 'Sacramento', 'Carson City', 'Phoenix', 'Santa Fe', 'Oklahoma City', 'Austin', 'Baton Rouge', 'Jackson', 'Little Rock', 'Jefferson City', 'Springfield', 'Indianapolis', 'Frankfort', 'Nashville']

**Total Distance of the Optimal Tour:**
175.4490

The optimal tour is also visualized on an interactive map.



**Reasoning**:
The previous command failed because the f-string was not closed properly. I need to regenerate the code block, ensuring the f-string is correctly defined to include the tour, total distance, and mention the map visualization.



In [32]:
# Format the results

results_summary = f"""
## Results

The Gurobi optimizer found the optimal tour for the Traveling Salesperson Problem covering the continental US capitals.

**Optimal Tour:**
{tour}

**Total Distance of the Optimal Tour:**
{total_distance:.4f}

The optimal tour is also visualized on an interactive map.
"""

print(results_summary)


## Results

The Gurobi optimizer found the optimal tour for the Traveling Salesperson Problem covering the continental US capitals.

**Optimal Tour:**
['Montgomery', 'Tallahassee', 'Atlanta', 'Columbia', 'Raleigh', 'Richmond', 'Annapolis', 'Dover', 'Trenton', 'Hartford', 'Providence', 'Boston', 'Augusta', 'Concord', 'Montpelier', 'Albany', 'Harrisburg', 'Charleston', 'Columbus', 'Lansing', 'Madison', 'Saint Paul', 'Des Moines', 'Topeka', 'Lincoln', 'Pierre', 'Bismarck', 'Cheyenne', 'Denver', 'Salt Lake City', 'Helana', 'Boise', 'Olympia', 'Salem', 'Sacramento', 'Carson City', 'Phoenix', 'Santa Fe', 'Oklahoma City', 'Austin', 'Baton Rouge', 'Jackson', 'Little Rock', 'Jefferson City', 'Springfield', 'Indianapolis', 'Frankfort', 'Nashville']

**Total Distance of the Optimal Tour:**
175.4490

The optimal tour is also visualized on an interactive map.



## Summarize conclusions

### Subtask:
Extract and summarize the main conclusions from the notebook.


**Reasoning**:
Extract and summarize the conclusions from the last markdown cell.



In [33]:
conclusions_cell = """
## Conclusions

The Traveling Salesman Problem (TSP) is the most popular combinatorial optimization problem. This problem is very easy to explain, although it is very complicated to solve. The largest TSP problem solved has 85,900 cities. The TSP is a source of discovery for new approaches to solve complex combinatorial optimization problems and has led to many applications.

In this modeling example, we have shown how to formulate the symmetric Traveling Salesman Problem as a MIP problem. We also showed how to dynamically eliminate subtours by using lazy constraints.
"""

# Summarize the conclusions
summarized_conclusions = """
## Conclusions

The Traveling Salesman Problem (TSP) is a widely recognized combinatorial optimization problem that is simple to describe but challenging to solve. It serves as a benchmark for developing new optimization techniques and has numerous real-world applications. This example demonstrated how to formulate the symmetric TSP as a Mixed-Integer Programming (MIP) problem and effectively solve it by dynamically eliminating subtours using lazy constraints within the Gurobi framework.
"""

print(summarized_conclusions)


## Conclusions

The Traveling Salesman Problem (TSP) is a widely recognized combinatorial optimization problem that is simple to describe but challenging to solve. It serves as a benchmark for developing new optimization techniques and has numerous real-world applications. This example demonstrated how to formulate the symmetric TSP as a Mixed-Integer Programming (MIP) problem and effectively solve it by dynamically eliminating subtours using lazy constraints within the Gurobi framework.



## Format as markdown

### Subtask:
Combine all the summarized sections into a single markdown-formatted text block.


**Reasoning**:
Combine all the summarized sections into a single markdown-formatted text block.



In [34]:
full_markdown_summary = f"""
# Traveling Salesperson Problem Example Summary

{summary}

{summary}

{implementation_explanation}

{subtour_explanation}

{results_summary}

{summarized_conclusions}
"""
print(full_markdown_summary)


# Traveling Salesperson Problem Example Summary


**Mathematical Model Formulation:**

**Sets and Indices:**
- `Capitals`: Set of US capital cities, indexed by `i` and `j`.
- `Pairings`: Set of allowed pairs of cities `(i, j)`.
- `S`: A subset of `Capitals`.
- `G`: A graph representing cities as nodes and pairings as edges.

**Parameters:**
- `d_i,j`: Distance between city `i` and city `j`. Distances are symmetric (`d_i,j = d_j,i`).

**Decision Variables:**
- `x_i,j`: Binary variable, 1 if city `i` is connected to city `j` in the tour, 0 otherwise.

**Objective Function:**
- Minimize the total distance of the tour: `Min Z = Sum(d_i,j * x_i,j) for all (i,j) in Pairings`.

**Constraints:**
- **Symmetry:** `x_i,j = x_j,i` for all `(i,j)` in Pairings (enforced implicitly in the Python code by aliasing variables).
- **Degree Constraint:** Each city must be connected to exactly two other cities: `Sum(x_i,j for all j connected to i) = 2` for all `i` in Capitals.
- **Subtour Elimination:** Fo

## Present markdown file content

### Subtask:
Provide the complete markdown content as a text output that the user can copy and save as a `.md` file.


**Reasoning**:
Print the complete markdown summary.



In [35]:
print(full_markdown_summary)


# Traveling Salesperson Problem Example Summary


**Mathematical Model Formulation:**

**Sets and Indices:**
- `Capitals`: Set of US capital cities, indexed by `i` and `j`.
- `Pairings`: Set of allowed pairs of cities `(i, j)`.
- `S`: A subset of `Capitals`.
- `G`: A graph representing cities as nodes and pairings as edges.

**Parameters:**
- `d_i,j`: Distance between city `i` and city `j`. Distances are symmetric (`d_i,j = d_j,i`).

**Decision Variables:**
- `x_i,j`: Binary variable, 1 if city `i` is connected to city `j` in the tour, 0 otherwise.

**Objective Function:**
- Minimize the total distance of the tour: `Min Z = Sum(d_i,j * x_i,j) for all (i,j) in Pairings`.

**Constraints:**
- **Symmetry:** `x_i,j = x_j,i` for all `(i,j)` in Pairings (enforced implicitly in the Python code by aliasing variables).
- **Degree Constraint:** Each city must be connected to exactly two other cities: `Sum(x_i,j for all j connected to i) = 2` for all `i` in Capitals.
- **Subtour Elimination:** Fo

## Summary:

### Data Analysis Key Findings

*   The Traveling Salesperson Problem (TSP) aims to find the shortest route visiting each city exactly once and returning to the origin. This example focuses on the symmetric TSP using Euclidean distances.
*   The problem is formulated as a Mixed-Integer Programming (MIP) model with binary decision variables indicating connections between cities, an objective to minimize total distance, and constraints ensuring each city has two connections.
*   Subtour elimination constraints are crucial to prevent invalid cycles that don't include all cities. Due to their exponential number, they are handled dynamically using lazy constraints and a callback function during the optimization process.
*   The Python implementation involves loading city data and coordinates, calculating pairwise Euclidean distances, creating a Gurobi model with decision variables and degree constraints, and implementing a callback function to detect and eliminate subtours.
*   The results section presents the optimal tour found by the optimizer and the total distance of this tour.

### Insights or Next Steps

*   The use of lazy constraints is a powerful technique for solving large-scale optimization problems with an exponential number of constraints, making otherwise intractable problems solvable.
*   Extending this example to the asymmetric TSP would require a different formulation of decision variables and constraints, as the distance between cities would not be symmetric.
